In [1]:
from IPython.core.display import display, HTML
from IPython.display import clear_output
display(HTML("<style>.container { width:90% }</style>"))
# ------------------------------------------------------------------

import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

# -- sklearn modules 
from sklearn.model_selection import train_test_split   #- partition train/test split 
from sklearn.tree import DecisionTreeClassifier        #- create a decison tree 
from sklearn.tree import export_text                   #- export tree as text rules 

# -- visualize decision trees and decision surfaces 
import graphviz 

# -- need this to render charts in notebook -- 
%matplotlib inline


In [2]:
df = pd.read_csv("./data/adult.csv")
df.head()

,age,workclass,fnlwgt,education,educational-num,marital-status,occupation,relationship,race,gender,capital-gain,capital-loss,hours-per-week,native-country,income
0,25,Private,226802,11th,7,Never-married,Machine-op-inspct,Own-child,Black,Male,0,0,40,United-States,<=50K
1,38,Private,89814,HS-grad,9,Married-civ-spouse,Farming-fishing,Husband,White,Male,0,0,50,United-States,<=50K
2,28,Local-gov,336951,Assoc-acdm,12,Married-civ-spouse,Protective-serv,Husband,White,Male,0,0,40,United-States,>50K
3,44,Private,160323,Some-college,10,Married-civ-spouse,Machine-op-inspct,Husband,Black,Male,7688,0,40,United-States,>50K
4,18,?,103497,Some-college,10,Never-married,?,Own-child,White,Female,0,0,30,United-States,<=50K


In [15]:
import random 
def set_nulls(df, pct_missing):
    # -1 because last col will always be y
    def _insert_random_null(x):
        x[random.randint(0, len(x) - 2)] = np.nan
        return x
    
    df = df.copy()
    sample_index = df.sample(frac=pct_missing).index 
    df.loc[sample_index] = df.loc[sample_index].apply(_insert_random_null, axis=1)
    return df

In [16]:
df1 = set_nulls(df,0.002)
df1.head()

,age,workclass,fnlwgt,education,educational-num,marital-status,occupation,relationship,race,gender,capital-gain,capital-loss,hours-per-week,native-country,income
0,25.0,Private,226802.0,11th,7.0,Never-married,Machine-op-inspct,Own-child,Black,Male,0.0,0.0,40.0,United-States,<=50K
1,38.0,Private,89814.0,HS-grad,9.0,Married-civ-spouse,Farming-fishing,Husband,White,Male,0.0,0.0,50.0,United-States,<=50K
2,28.0,Local-gov,336951.0,Assoc-acdm,12.0,Married-civ-spouse,Protective-serv,Husband,White,Male,0.0,0.0,40.0,United-States,>50K
3,44.0,Private,160323.0,Some-college,10.0,Married-civ-spouse,Machine-op-inspct,Husband,Black,Male,7688.0,0.0,40.0,United-States,>50K
4,18.0,?,103497.0,Some-college,10.0,Never-married,?,Own-child,White,Female,0.0,0.0,30.0,United-States,<=50K


In [17]:
df1.describe(include='all').T.sort_values('unique')

,count,unique,top,freq,mean,std,min,25%,50%,75%,max
gender,48836,2,Male,32644,NaN,NaN,NaN,NaN,NaN,NaN,NaN
income,48842,2,<=50K,37155,NaN,NaN,NaN,NaN,NaN,NaN,NaN
race,48829,5,White,41750,NaN,NaN,NaN,NaN,NaN,NaN,NaN
relationship,48835,6,Husband,19713,NaN,NaN,NaN,NaN,NaN,NaN,NaN
marital-status,48834,7,Married-civ-spouse,22375,NaN,NaN,NaN,NaN,NaN,NaN,NaN
workclass,48834,9,Private,33900,NaN,NaN,NaN,NaN,NaN,NaN,NaN
occupation,48831,15,Prof-specialty,6172,NaN,NaN,NaN,NaN,NaN,NaN,NaN
education,48838,16,HS-grad,15784,NaN,NaN,NaN,NaN,NaN,NaN,NaN
native-country,48834,42,United-States,43826,NaN,NaN,NaN,NaN,NaN,NaN,NaN
age,48836,NaN,NaN,NaN,38.6438,13.711,17,28,37,48,90


In [18]:
df1.isnull().sum(axis = 0)

age                 6
workclass           8
fnlwgt              6
education           4
educational-num     3
marital-status      8
occupation         11
relationship        7
race               13
gender              6
capital-gain        5
capital-loss        7
hours-per-week      6
native-country      8
income              0
dtype: int64

In [19]:
df1.to_csv("./data/adult2.csv", index=False)